In [1]:
import numpy as np
from shapely.geometry import box
from supervision.detection.utils import box_iou_batch
import random

# 1. Intersection over Union (IoU)

In [ ]:
def yolo_to_corners(x_center, y_center, width, height):
    x_min = x_center - width / 2
    y_min = y_center - height / 2
    x_max = x_center + width / 2
    y_max = y_center + height / 2
    return x_min, y_min, x_max, y_max

def iou_shapely(box1, box2):
    x1_min, y1_min, x1_max, y1_max = yolo_to_corners(*box1)
    x2_min, y2_min, x2_max, y2_max = yolo_to_corners(*box2)

    box1_shapely = box(x1_min, y1_min, x1_max, y1_max)
    box2_shapely = box(x2_min, y2_min, x2_max, y2_max)

    intersection_area = box1_shapely.intersection(box2_shapely).area
    union_area = box1_shapely.union(box2_shapely).area

    iou = intersection_area / union_area #if union_area > 0 else 0
    return iou

In [ ]:
box1 = (0.5, 0.5, 0.4, 0.4)  
box2 = (0.55, 0.65, 0.4, 0.4)  

iou_s = iou_shapely(box1, box2)
print(f"Shapely IoU : {iou_s:.4f}")


# Supervision lib takes xmin...ymax
box1_corners = np.array([yolo_to_corners(*box1)]) 
box2_corners = np.array([yolo_to_corners(*box2)])

iou_supervision = box_iou_batch(box1_corners, box2_corners)[0, 0]
print(f"Supervision IoU : {iou_supervision:.4f}")

Shapely IoU : 0.3763
Supervision IoU : 0.3763


In [4]:
# Both provide same results

# 2. Average Precision (AP)

### a. Pascal VOC 11 point interpolation method
* 11 treshold points bw [0, 1] for recall (0.1 diff).
* For each recall threshold, take the maximum precision at or above that recall level.
* Average of all at last for AP

In [5]:
def voc_11_point_ap(precision, recall):
    recall_thresholds = np.linspace(0, 1, 11) 
    precision_interp = []
    for th in recall_thresholds:
        max_precision = np.max(precision[recall >= th]) if np.any(recall >= th) else 0
        # print(max_precision)
        precision_interp.append(max_precision)
        
    return np.mean(precision_interp)

In [6]:
precision = np.array([1.00, 1.00, 0.67, 0.75, 0.80, 0.71])
recall = np.array([0.20, 0.40, 0.40, 0.60, 0.80, 1.00])

print(voc_11_point_ap(precision=precision, recall=recall))

0.8745454545454546


### b. COCO 101-point interpolation method
* Same approach as VOC 11, only difference is recall threshold count. Here it is 101.

In [7]:
def coco_101_point_ap(precision, recall):
    recall_thresholds = np.linspace(0, 1, 101)
    precision_interp = []
    for th in recall_thresholds:
        max_precision = np.max(precision[recall >= th]) if np.any(recall >= th) else 0
        precision_interp.append(max_precision)

    return np.mean(precision_interp)

In [8]:
print(coco_101_point_ap(precision=precision, recall=recall))

0.8633663366336631


###  c. Area under Precision-Recall Curve (AP) 
* Integration (auc)

In [9]:
def ap_pr_curve(precision, recall):
    sorted_indices = np.argsort(recall)
    recall = np.array(recall)[sorted_indices]
    precision = np.array(precision)[sorted_indices]
    ap = np.trapz(precision, recall)
    
    return ap

In [10]:
print(ap_pr_curve(precision, recall))

0.6479999999999999


---------

### Randomly generate 10 images and compare AP50

In [29]:
def generate_random_box(image_size=100, box_size=20):
    x_center = random.randint(box_size // 2, image_size - box_size // 2)
    y_center = random.randint(box_size // 2, image_size - box_size // 2)
    # print(x_center, y_center)
    return (x_center, y_center, box_size, box_size)

def compute_precision_recall(gt_boxes, pred_boxes, iou_threshold=0.5):
    ious = np.array([[iou_shapely(gt, pred) for pred in pred_boxes] for gt in gt_boxes])

    tp, fp, fn = 0, 0, len(gt_boxes)
    precisions, recalls = [], []
    for i in range(len(pred_boxes)):
        max_iou = np.max(ious[:, i])
        if max_iou >= iou_threshold:
            tp += 1
            fn -= 1
        else:
            fp += 1
        precision = tp / (tp + fp) 
        recall = tp / (tp + fn)
        precisions.append(precision)
        recalls.append(recall)

    return np.array(precisions), np.array(recalls)

In [32]:
def evaluate_ap_methods():
    num_images = 10
    gt_boxes = [generate_random_box() for _ in range(num_images)]
    pred_boxes = [generate_random_box() for _ in range(num_images)]
    
    precision, recall = compute_precision_recall(gt_boxes, pred_boxes)

    ap_voc = voc_11_point_ap(precision, recall)
    ap_coco = coco_101_point_ap(precision, recall)
    ap_pr = ap_pr_curve(precision, recall)

    print(f"Pascal VOC 11-point AP: {ap_voc:.4f}")
    print(f"COCO 101-point AP: {ap_coco:.4f}")
    print(f"PR Curve Area AP: {ap_pr:.4f}")

evaluate_ap_methods()

Pascal VOC 11-point AP: 0.2182
COCO 101-point AP: 0.1782
PR Curve Area AP: 0.0586
